In [ ]:
!pip install llama-index
!pip install gitpython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex, Document
import requests         # for api calls
import pandas as pd     # for handling data
import csv
import git
from bs4 import BeautifulSoup
import pdfplumber
from io import StringIO  # To treat the raw CSV data as a file-like object
#from sentence_transformers import SentenceTransformer
#from sklearn.metrics.pairwise import cosine_similarity

In [46]:
#parsing data from git repo
repo_url = "https://raw.githubusercontent.com/tnguyen21/caseworker-agent/refs/heads/main/Section8-Resources/Section_8_Housing_Resources%20-%20Links.csv"

#using requests to access the raw repo
response = requests.get(repo_url)
if response.status_code == 200:
  csv_data = response.text
else:
  print(f"Failed to retrieve data. Status code: {response.status_code}")

#using StringIO to treat csv_data as a file
csv_file = StringIO(csv_data)

#reading the csv_file
reader = csv.DictReader(csv_file)
#df

#extracting just the links and appending to a list
links = []
for link in reader:
  links.append(link['Url'])
links

#fetching content from extracted links
def fetch_content_from_url():
  documents = []

  for link in links:
    response = requests.get(link)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      print(soup.prettify())

      # creating Document object
      doc = Document(text=soup.prettify(), metadata={"url":link})
      documents.append(doc)
    else:
      print(f"Failed to retrieve data. Status code: {response.status_code}")

  return documents

  # Function to create and save the index
def build_index(documents):
    """
    Create an index with the given documents and save it to disk.
    """
    print("Building index...")

  # Create the index with the provided documents
    index = GPTVectorStoreIndex(documents)
    query_engine = index.as_query_engine()
    response = query_engine.query("Where can I find housing vouchers?")
    print("Query Response:", response)
    return index

documents = fetch_content_from_url()  # Fetch and create documents from the URLs
build_index(documents)


<!DOCTYPE html>
<!--[if lt IE 9]><html lang="en" class="no-js lt-ie10 lt-ie9"><![endif]-->
<!--[if IE 9]><html lang="en" class="no-js is-ie9 lt-ie10"><![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en">
 <!--<![endif]-->
 <head>
  <title>
   Housing Resources : Resources : The Arc : SpArc Philadelphia
  </title>
  <link href="https://cdn.firespring.com/images/94eb5afe-6621-41d6-834f-461890066a68" rel="shortcut icon"/>
  <!-- Meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Housing Resources : Resources : The Arc : SpArc Philadelphia" property="og:title"/>
  <meta content="https://arcphiladelphia.org/arc_philadelphia/resourcessection/housing-resources.html" property="og:url"/>
  <meta content="website" property="og:type"/>
  <meta content="summary" name="twitter:card"/>
  <meta content="Housing Resources : Resources : The Arc : SpArc Philadelphia" name="twitter:title"/>
  <link href="//cdn.fi

TypeError: Document.__init__() missing 1 required positional argument: 'page_content'

Separate

In [49]:
# Fetch all files in the GitHub repository
def get_github_repo_files(repo_owner, repo_name, branch="main"):
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents?ref={branch}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to list files. Status code: {response.status_code}")
        return []

# Process CSV files
def process_csv(content):
    csv_file = StringIO(content)
    reader = csv.DictReader(csv_file)

    documents = []
    for row in reader:
        text_content = f"URL: {row['Url']}, Description: {row.get('Description', 'No description available')}"
        doc = Document(text=text_content, metadata={"url": row['Url']})
        documents.append(doc)

    return documents

# Process PDF files
def process_pdf(content):
    with pdfplumber.open(content) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

    doc = Document(text=text, metadata={"file_type": "pdf"})
    return doc

# Process HTML files from URL
def process_html_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()  # or soup.prettify() if you want HTML

        doc = Document(text=text_content, metadata={"url": url})
        return doc
    else:
        print(f"Failed to fetch content from {url}. Status code: {response.status_code}")
        return None

# Main function to process all files
def process_files_from_repo(repo_owner, repo_name):
    files = get_github_repo_files(repo_owner, repo_name)
    documents = []

    for file in files:
        file_name = file['name']
        file_url = file['download_url']
        file_type = file['name'].split('.')[-1].lower()  # Get the file extension

        print(f"Processing file: {file_name}")

        if file_type == 'csv':
            content = requests.get(file_url).text
            documents.extend(process_csv(content))

        elif file_type == 'pdf':
            content = requests.get(file_url, stream=True).raw  # Download as raw stream for PDF
            documents.append(process_pdf(content))

        elif file_type == 'html':
            documents.append(process_html_from_url(file_url))

        else:
            print(f"Skipping unsupported file type: {file_name}")

    return documents

# Build and query the index
def build_index(documents):
    print("Building index...")
    index = GPTVectorStoreIndex(documents)
    return index

# Main execution
if __name__ == "__main__":
    repo_owner = "tnguyen21"
    repo_name = "caseworker-agent"

    documents = process_files_from_repo(repo_owner, repo_name)
    if documents:
        # Create the index
        index = build_index(documents)

        # Query the index (example query)
        query_engine = index.as_query_engine()
        response = query_engine.query("Where can I find housing vouchers?")
        print("Query Response:", response)
    else:
        print("No documents found to index.")


Processing file: .dockerignore
Skipping unsupported file type: .dockerignore
Processing file: .gitignore
Skipping unsupported file type: .gitignore
Processing file: Data
Skipping unsupported file type: Data
Processing file: Dockerfile
Skipping unsupported file type: Dockerfile
Processing file: LICENSE
Skipping unsupported file type: LICENSE
Processing file: README.md
Skipping unsupported file type: README.md
Processing file: Scripts
Skipping unsupported file type: Scripts
Processing file: Section8-Resources
Skipping unsupported file type: Section8-Resources
Processing file: backend
Skipping unsupported file type: backend
Processing file: fly.toml
Skipping unsupported file type: fly.toml
Processing file: frontend
Skipping unsupported file type: frontend
Processing file: package-lock.json
Skipping unsupported file type: package-lock.json
Processing file: prompts
Skipping unsupported file type: prompts
Processing file: requirements.txt
Skipping unsupported file type: requirements.txt
Proc